# Time Serise Algorithm for Sentinel-2 Cloud Mask

## Overview
This product is a time series cloud and cloud shadow detection algorithm for Sentinel-2 surface reflectance data.It models time series of surface reflectance derived indices and calculates time series abnormality coefficients for pixels in the time series. It does not rely on predefined training data to generate complex models with many rule sets, which often work well for data similar to the training data while returning poor results for data contrasting to the training data. Instead, it identifies cloud and cloud shadows by detecting local abnormalities in temporal and spatial contexts from abnormality coefficients.


## Required Modules 

### DEA modules


The notebook requires functions in DEA Datacube API. To load DEA module, open a terminal and run following commands: 

`module use /g/data/v10/public/modules/modulefiles`

`module load dea`

### Other Python modules 

DEADataHandling is a module written by scientists and engineers from DEA teams. The module consists of a number of wrapper functions to handle various DEA datasets. The module can be download at https://github.com/GeoscienceAustralia/dea-notebooks. 

## Input Data

Sentinel-2 NBART time series data in 20m x 20m resolution.


## Workflow of the Program  

### Data loading

The program call Function F1 to load S2 NBART time series data in a Xarray dataset, create tsmask dataarray  


### Time Series Cloud and Cloud shadow detection

The program calls Function F7 to perform time series cloud/shadow detection for one pixel. 

### Spatial Filter

The program calls Function F10 to filter out isolated / single cloud and shadow pixels. 

### Spatial Buffer

The program calls Function F3 to extend cloud/shadow detection coverage by one pixel buffers 

## Output Data

The program produces a cloud mask data with the same dimension as the input time series data,a Sentinel-2 pixel is calsiified as one of four distinctive categories:
 

No observation ---> 0
Clear ---> 1
Cloud ---> 2
Cloud shadow ---> 3


The S2 NBARt time series and tsmask are saved as ENVI images (Function F9) and NetCDF files. 


**Function ID: F1** 


**Function Name:**

load_s2_nbart_ts

**Description:** 


This function loads Sentinel-2 surface reflectance data from DEA database. The spatial coverage is specfied by a rectangle bounding box. The temporal coverage is specified by the start date and the end date of the time series data

**Input:**

The lat./Lon. of the top-left and bottom-right corners of a rectangle bounding box, he start date and the end date of the time series data 

**Return:**
an Xarray contains 6 spectral bands of time series surface reflectance data 

**Called by:**
Main Program

In [1]:
## Function F1

def load_s2_nbart_ts(dc, lat_top, lat_bottom, lon_left, lon_right, start_of_epoch, end_of_epoch):
    
    # Define spatial and temporal coverage 
    
    newquery={'x': (lon_left, lon_right),
          'y': (lat_top, lat_bottom),
          'time': (start_of_epoch, end_of_epoch),
          'output_crs': 'EPSG:3577',
          'resolution': (-20, 20)} 
    
    # Names of targeted spectral bands
    
    allbands=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_2', 'nbart_swir_2', 'nbart_swir_3']

    # Band names used with in the dataset 
    new_bandlabels = ['blue','green', 'red', 'nir', 'swir1', 'swir2']

    # Load S2 data using Datacube API
    s2_ds = DEADataHandling.load_clearsentinel2(dc=dc, query=newquery, sensors=('s2a', 's2b'), product='ard',
                                    bands_of_interest=allbands, mask_pixel_quality=False, 
                                    mask_invalid_data=False, masked_prop=0.0)

    # Rename spectral band names to new band labels
 
    rndic=dict(zip(allbands, new_bandlabels))
    s2_ds = s2_ds.rename(rndic)
    
    # Add tsmask dataarray to the dataset
    
    s2_ds['tsmask']=s2_ds['blue']
    s2_ds['tsmask'].values=np.zeros((s2_ds['time'].size, s2_ds['y'].size,s2_ds['x'].size), dtype=np.uint8)
    

    
    
    return s2_ds

**Function ID: F6** 

**Function Name:**

findnbsa(vsa, k, N, dws_flags, vtsmask)

**Description:** 


This function find mean values of neighbour pixels of a specified segment in a time series

**Input:**

vsa: 1D array, time series
k: location of the specified segment
N: length of the segment
dws_flags: flags indicating that a pixel is either a non-shadow pixel or a water pixel
vtsmask: time series of cloud/shadow labels

**Return:**

mean values of neighbour pixels of the specified segment

**Called by:**
Fuction F5

In [2]:
#Fuction F6

# Given the length of the segment is N, the function will find N clear pixels closest to the segment from the left, N
# N clear pixels closest to the segment from the right, calculate the mean of these 2N pixels

def findnbsa(vsa, k, N, vss, dws_flags, vtsmask):
    
    #clear pixel counter
    cc=0
    
    # Search direction, 0 -- search the left, 1 -- search the right  
    dr=0
    
    # Directional flags, 0 -- search can be continued, 1 -- search reach the boundary  
    mvd=[0,0]
    
    # location of the left of the segment
    lpt=k
    
    # location of the right of the segment
    rpt=k+N-1
    
    # sum of the found clear pixels 
    mid=0.0
    
    while cc<2*N:
        
        # search the left
        if (dr==0):
            # Not reach the left boundary yet
            if (mvd[0]==0):
                while True:
                    # Move the left pointer to 1 pixel left
                    lpt -= 1
                    # reach the begining of the time series?
                    if (lpt<0):
                        # Yes, modify the directional flags, change the srach directional
                        mvd[dr]=1
                        dr=1
                        break
                    elif (vtsmask[lpt]==1 and dws_flags[lpt]==1):
                        # No, if the pixel is a clear pixels and the pixel is not a potenial shadow pxiels
                        # Add the value of the pixel to the sum of the found clear pixels
                        mid += vsa[lpt]
                        # update the clear pixel counter, change the search direction to right 
                        cc += 1
                        dr=1
                        break
            else:
                dr=1
        else: 
            # search the right
            if (mvd[1]==0):
                # Not reach the right boundary yet
                while True:
                    # Move the right pointer to 1 pixel right
                    rpt += 1
                    # reach the end of the time series?
                    if (rpt==vss):
                        # Yes, modify the directional flags, change the srach directional 
                        mvd[dr]=1
                        dr=0
                        break
                    elif (vtsmask[rpt]==1 and dws_flags[rpt]==1):
                        # No, if the pixel is a clear pixels and the pixel is not a potenial shadow pxiels
                        # Add the value of the pixel to the sum of the found clear pixels
                        mid += vsa[rpt]
                        # update the clear pixel counter, change the search direction to left
                        cc += 1
                        dr=0
                        break
            else:
                dr=0

        # The search reach the boundaries in both direction, exit the search
        if (mvd[0]==1 and mvd[1]==1):
            break
            
    # if not enough clear pixels found, return 0, otherwise return the mean of the found 2N clear pixels
    if (cc<2*N):
        return 0
    else:
        return mid/(2*N)
    
    
        
    


**Function ID: F5** 

**Function Name:**

testpair(sa, dwi, N, tsmask)

**Description:** 


This function identifies cloud and shadow pixels in a time series by comparing its value to its neighbours

**Input:**

sa: 1D numpy array, time series of the mean of surface reflectance value of the 6 spectral bands
dwi: 1D numpy array, time series of MNDWI, modified normalised water difference index
tsmasak: time series of cloud/shadow labels

**Return:**

updated tsmask 

**Called by:**

Function F7

In [3]:
# Function F5

def testpair(sa, dwi, N, tsmask):
    
    
    # cloud detection threshold, the lower the value, the higher cloud detection rate  
    cspkthd = 0.42
    
    # shade detection threshold, the lower the value, the higher shade detection rate 
    sspkthd = 0.42

    # the minimum theshold of a cloud pixel, i.e., all cloud pixels will have a band average 
    # value higher that this theshold
    cloudthd = 0.10
    
    # The shadow pixel theshold
    shadowthd = 0.055


    
    # Find all clear pixels in the time series
    
    validx=np.where(tsmask==1)[0]
    
    # The number of the clear pixels in the time series
    vss=validx.size
    
    # Not enough clear pixels in the time series
    if vss<3*N:
        return
   
    # Filter out invalid, cloud, shadow points in time series
    vsa=sa[validx]
    vdwi=dwi[validx]
    vtsmask=tsmask[validx]
    
    #flags which indicates if 
    chmarker=np.zeros(vss, dtype=np.int8)
    
    # flags which indicates a pixel is either a non-shadow or a water pixels
    dws_flags=np.logical_or(vsa>shadowthd, vdwi>0)
 
    #Total number of segments in the time series
    numse=vss-N+1
    
    # array to store mean of the segments
    msa=np.zeros(numse, dtype=np.float32)
    
    #calculate mean values of the time series segments
    
    if (N==1):
        msa=vsa
        
    else:
        for i in np.arange(numse):
            msa[i]=vsa[i:i+N].sum()/N

            
    
    #sort the time series of mean of the segemnts
    sts = np.argsort(msa)
    
    # reverse the order from ascending to descending, so that sts contains index number of msa array, from 
    # highest values to the lowest
    
    sts = sts[::-1]
    
    
    for k in sts:
        
        if (chmarker[k]==0):
            # mean of the segment
            m2=msa[k]
            # mean of the neighbouring 2N pixels
            mid=findnbsa(vsa, k, N, vss, dws_flags, vtsmask)
            
            # check if the mean of segemnt is significantly different from the neighbouring pixels
            if (m2>mid and mid>0):
                if ((m2-mid)/mid> cspkthd and m2>cloudthd):
                    #cloud pixels
                    vtsmask[k:k+N]=2
                    chmarker[k:k+N]=1
                    
            elif (mid>m2 and mid>0):
                if ((mid-m2)/m2> sspkthd and m2<shadowthd):
                    #shadow pixels
                    vtsmask[k:k+N]=3
                    chmarker[k:k+N]=1

            
         
    #update the orginal time series mask
    tsmask[validx]=vtsmask

**Function ID: F7** 

**Function Name:**

perpixel_filter_direct

**Description:** 


This function performs time series cloud/shadow detection for one pixel 

**Input:**

Surface reflectance time series data of band blue, green, red, nir, swir1, swir2 for the pixel 


**Return:**

Updated cloud/shadow time series data for the pixel


**Called by:**

Main program

In [4]:
## Function F7

def perpixel_filter_direct(blue, green, red, nir, swir1, swir2, tsmask):
    
    scale=10000.0
    ivd=-999/scale
    
    tsmask[:] = 1
    
    blue = blue.copy().astype(np.float32) / scale
           
    green = green.copy().astype(np.float32) / scale
    
    red = red.copy().astype(np.float32) / scale
        
    nir = nir.copy().astype(np.float32) / scale
            
    swir1 = swir1.copy().astype(np.float32) / scale
        
    swir2 = swir2.copy().astype(np.float32) / scale
        
    # detect pixels with invalid data value
    
    
    tsmask[blue==ivd]=0
    
    tsmask[green==ivd]=0
    
    tsmask[red==ivd]=0
    
    tsmask[nir==ivd]=0
    
    tsmask[swir1==ivd]=0
    
    tsmask[swir2==ivd]=0
      
    # calculate indices
    
    # mean of 6 spectral bands
    
    sa=(blue+green+red+nir+swir1+swir2)/6
    
    #modified normalised difference water index
    
    mndwi=(green-swir1)/(green+swir1)
    
    #modified soil adjusted vegetation index
    
    msavi=(2*nir+1-np.sqrt((2*nir+1)*(2*nir+1)-8*(nir-red)))/2
    
    # Band different ratio between red and blue 
    wbi=(red-blue)/blue
    
    # Sum of red and blue band
    rgm=red+blue
    
    # Band different ratio between green and mean of red and blue
    grbm=(green-(red+blue)/2)/((red+blue)/2)
    
    #Bright cloud theshold 
    maxcldthd=0.45
    
    # label all ultra-bright pixels as clouds
    tsmask[sa>maxcldthd]=2
    
    
   
    # detect single cloud / shadow pixels
    testpair(sa, mndwi, 1, tsmask)
    testpair(sa, mndwi, 1, tsmask)
    testpair(sa, mndwi, 1, tsmask)
    
    # detect 2 consecutive cloud / shadow pixels
    testpair(sa, mndwi, 2, tsmask)
    testpair(sa, mndwi, 2, tsmask)
    
    # detect 3 consecutive cloud / shadow pixels
    testpair(sa, mndwi, 3, tsmask)
    
    
    # detect single cloud / shadow pixels
    testpair(sa, mndwi, 1, tsmask)
    
    

    
    #cloud shadow theshold
    shdthd = 0.05;

    
    # mndwi water pixel theshold
    dwithd=-0.05

    # mndwi baregroud pixel theshold
    landcloudthd=-0.38

    # msavi water pixel theshold
    avithd=0.06
    
     # mndwi water pixel theshold
    wtdthd=-0.2


    
    for i, lab in enumerate(tsmask): 
        
        if (lab==3 and mndwi[i]>dwithd and sa[i]< shdthd): #water pixel, not shadow
            tsmask[i] = 1
            
        if (lab==2 and mndwi[i]<landcloudthd): # bare ground, not cloud
            tsmask[i] = 1
        
        if (lab==3 and msavi[i]<avithd and mndwi[i]>wtdthd): # water pixel, not shadow
            tsmask[i] = 1

        if (lab==1 and wbi[i]<-0.02 and rgm[i]>0.06 and rgm[i]<0.29 and mndwi[i]<-0.1 and grbm[i] < 0.2): # thin cloud
            tsmask[i] = 2
            

    
    return tsmask

**Function ID: F8** 



**Function Name:**

create_ts_tuples

**Description:** 


This function creates a time series of tuples of Sentinel-2 surface reflectance data


**Input:**

 An xarray dataset contains 6 spectral bands of time series surface reflectance data 

**Return:**

None. The tsmask in the input xarray dataset will be updated with cloud/shadow mask values  

**Called by:**
Main program

In [5]:
#Function F8

def create_ts_tuples(s2_ds):
    
     
    irow = s2_ds['y'].size
    icol = s2_ds['x'].size
    pnum = irow*icol
    
    
    ts_tuples = []
    
    for i in np.arange(pnum):
        
        y = int(i/icol)
        x = i%icol
        
        # copy time series spectral data from the data set, scale the data to float32, in range (0, 1.0)
    
        blue = s2_ds['blue'].values[:, y, x]
           
        green = s2_ds['green'].values[:, y, x]
    
        red = s2_ds['red'].values[:, y, x]
        
        nir = s2_ds['nir'].values[:, y, x]
            
        swir1 = s2_ds['swir1'].values[:, y, x]
        
        swir2 = s2_ds['swir2'].values[:, y, x]
        
        tsmask = s2_ds['tsmask'].values[:, y, x]
      
        ts_tuples.append((blue, green, red, nir, swir1, swir2, tsmask))

        
    return ts_tuples

**Function ID: F4** 

**Function Name:**

write_multi_time_dataarray_v2

**Description:** 


This function outputs an dataarray as an ENVI image 

**Input:**

filename: The output filename 
dataarray: The output dataarray 
xs: number of columns
ys: number of rows
profile_override:  profile name of output format

**Return:**

None


**Called by:**

Function F9

In [6]:
#Function F4
def write_multi_time_dataarray_v2(filename, dataarray, xs, ys, **profile_override):
    profile = {
        'width': xs,
        'height': ys,
        'transform': dataarray.affine,
        'crs':'EPSG:3577',
        'count': len(dataarray.time),
        'dtype': str(dataarray.dtype)
    }
    profile.update(profile_override)

    with rasterio.open(str(filename), 'w', **profile) as dest:
        for time_idx in range(len(dataarray.time)):
            bandnum = time_idx + 1
            dest.write(dataarray.isel(time=time_idx).data, bandnum)
            


**Function ID: F2** 

**Function Name:**

get_timebandnames

**Description:** 


This function creates a list of date strings from the time series of date and time in the dataset

**Input:**

s2 _ds: the dataset

**Return:**

a list of date strings


**called by:**

Function F9

In [7]:
#Function F2

def get_timebandnames(s2_ds):
    timelist = []
    for t in s2_ds['time']:
        timelist.append(time.strftime("%Y-%m-%d", time.gmtime(t.astype(int)/1000000000)))
    
    return timelist

**Function ID: F9** 



**Function Name:**

output_ds_to_ENVI(bandsets, dirc, s2_ds)

**Description:** 


This function outputs a set of dataarrays in a dataset as ENVI images



**Input:**

bandsets: specified the names of the dataarray in the dataset to be saved
dirc: the directory where the image files are save
s2_ds: the dataset 

**Return:**

None. 

**Called by:**
Main program

In [8]:
#Function F9

def output_ds_to_ENVI(bandsets, dirc, s2_ds):
    
    # Create a list of date strings from the time series of time in the dataset
    
    timebandnames = get_timebandnames(s2_ds)
    
    xs= s2_ds['x'].size
    ys= s2_ds['y'].size
    
    for bandname in bandsets:
        banddata=s2_ds[bandname]   
        filename=dirc+'/NBAR_'+bandname+'.img'
            
        # output dataarray as ENVI image file
        
        write_multi_time_dataarray_v2(filename, banddata, xs, ys, driver='ENVI')
     
            
        # Update ENVI header files with  the list of the date strings 
        hdrfilename = dirc+'/NBAR_'+bandname+'.hdr'
        h = envi.read_envi_header(hdrfilename)
        h['band names']=timebandnames
        envi.write_envi_header(hdrfilename, h)

**Function ID: F10** 


**Function Name:**

spatial_filter

**Description:** 


This function labels cloud and shadow pixels with less than M surrounding cloud/shadow pixels as clear pixels
 

**Input:**

One scene of the tsmask dataarray

**Return:**

updated tsmask with cloud/shadow mask values  

**Called by:**
Main program

In [9]:
#Function F10

def spatial_filter(onescene):
    
    tsmask=onescene
    M = 2
    
    irow, icol = onescene.shape
    for y in np.arange(irow-2):
        for x in np.arange(icol-2):
            block=tsmask[y:y+3, x:x+3]
            # if the center pixel in the block is a cloud or shadow
            if (block[1,1]==2 or block[1,1]==3):
                # if total number of cloud/shadow pixels in the block is less than M+1,
                # label the center pixel as a clear pixel
                
                if (np.logical_or(block==2, block==3).sum()<M+1):
                    tsmask[y+1, x+1] = 1
     
    return tsmask

**Function ID: F3** 


**Function Name:**

spatial_buffer

**Description:** 


This function labels neighbouring pixels of cloud and shadow pixels as cloud/shadow pixels
 

**Input:**

One scene of the tsmask dataarray

**Return:**

updated tsmask with cloud/shadow mask values  

**called by:**
Main program

In [10]:
#Function F3

def spatial_buffer(onescene):
    
    tsmask=onescene
    newmask=onescene.copy()
    
    
    
    irow, icol = onescene.shape
    for y in np.arange(irow-2):
        for x in np.arange(icol-2):
            block=tsmask[y:y+3, x:x+3]
            # if the center pixel in the block is a cloud or shadow
            if (block[1,1]==2 or block[1,1]==3):
                # label neighbouring pixels as cloud/shadow pixels
                newmask[y:y+3, x:x+3]=block[1,1]
    
   
    # Only change clear pixels' labels            
    vidx=(tsmask==1)
    tsmask[vidx]=newmask[vidx]
    
    return tsmask




In [11]:
## Start Main program

## Import modules 

import datacube
import sys
import numpy as np
import time
import os
import rasterio
import xarray as xr

import DEADataHandling

from multiprocessing import Pool 
import rasterio
from spectral import envi



/g/data/v10/public/modules/dea/20200819/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


In [12]:
## Specify input parameters

#lat_top, lat_bottom, lon_left, lon_right =  -35.144, -35.505, 148.985, 149.284
lat_top, lat_bottom, lon_left, lon_right =  -35.244, -35.344, 149.055, 149.155
start_of_epoch, end_of_epoch = '2017-01-01', '2018-12-31'
  


In [13]:
# Load surface reflectance data

dc = datacube.Datacube(app='load_clearsentinel')


# Call Function F1
# Load Sentinel-2 data 

s2_ds = load_s2_nbart_ts(dc, lat_top, lat_bottom, lon_left, lon_right, start_of_epoch, end_of_epoch)



Loading s2a pixel quality
    Loading 70 filtered s2a timesteps
Loading s2b pixel quality
    Loading 54 filtered s2b timesteps
Combining and sorting s2a, s2b data


In [14]:
# create a list of tuples as input of the cloud detection functions
# startmap method of the Pool class from Multiprocessing module requires an ierative object for function parameters

ts_tuples = create_ts_tuples(s2_ds)


In [15]:
%%time

results=[]

# number of process for the  pool object
number_of_workers=8
# Create a Pool object with a number of processes
p=Pool(number_of_workers)
# Start runing the cloud detection function using a pool of independent processes
results=p.starmap(perpixel_filter_direct, ts_tuples)
# Finish the parallel runs
p.close()
# Join the results and put them back in the correct order
p.join()

/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
/g/data/v10/public/modules/dea-env/20200713/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning

CPU times: user 24.6 s, sys: 4.83 s, total: 29.4 s
Wall time: 23min 30s


In [16]:
# Save the cloud/shadow masks to the 'tsmask' dataarray in the s2_ds dataset
irow = s2_ds['y'].size
icol = s2_ds['x'].size
for y in np.arange(irow):
    for x in np.arange(icol):
        s2_ds['tsmask'].values[:, y, x]=results[y*icol+x]

In [17]:
%%time

results=[]

# number of process for the  pool object
number_of_workers=8
# Create a Pool object with a number of processes
p=Pool(number_of_workers)

#create a list of scene
paralist=[ s2_ds['tsmask'].values[i, :, :] for i in np.arange(s2_ds.time.size)]
# Start runing the spatial filter function using a pool of indepedent processes
results=p.map(spatial_filter, paralist)
# Finish the parallel runs
p.close()
# Join the results and put them back in the correct order
p.join()

CPU times: user 197 ms, sys: 417 ms, total: 614 ms
Wall time: 1min 19s


In [18]:
# Save the cloud/shadow masks to the 'tsmask' dataarray in the s2_ds dataset
for i in np.arange(s2_ds.time.size):
    s2_ds['tsmask'].values[i, :, :] = results[i]

In [19]:
%%time

results=[]

# number of process for the  pool object
number_of_workers=8
# Create a Pool object with a number of processes
p=Pool(number_of_workers)

# Create a list of scene
paralist=[ s2_ds['tsmask'].values[i, :, :] for i in np.arange(s2_ds.time.size)]
# Start runing the spatial_buffer function using a pool of indepedent processes
results=p.map(spatial_buffer, paralist)
# Finish the parallel runs
p.close()
# Join the results and put them back in the correct order
p.join()

CPU times: user 100 ms, sys: 394 ms, total: 494 ms
Wall time: 42.5 s


In [20]:
# Save the cloud/shadow masks to the 'tsmask' dataarray in the s2_ds dataset
for i in np.arange(s2_ds.time.size):
    s2_ds['tsmask'].values[i, :, :] = results[i]

In [21]:
# Eliminate invalid attribute values

for key, variable in s2_ds.variables.items():
    if 'spectral_definition' in variable.attrs:
        del variable.attrs['spectral_definition']

In [22]:
## Output spectral data and tsmask as NetCDF file

dirc='/g/data/u46/pjt554/tmp/test.nc'
s2_ds.to_netcdf(dirc)

In [23]:
# Name of the dataarrays 
bandsets = ['blue','green', 'red', 'nir', 'swir1', 'swir2', 'tsmask']
#The directory where the files will be saved
dirc='/g/data/u46/pjt554/tmp'
# Output spectral and tsmask dataaray as ENVI images
output_ds_to_ENVI(bandsets, dirc, s2_ds)

In [24]:
## End Main program